In [4]:
!uv pip install 'langevals[all]' agno

Resolved 204 packages in 271ms                                       
⠙ Preparing packages... (0/1)                                                   
⠙ Preparing packages... (0/1)----     0 B/1.08 MiB                      
⠙ Preparing packages... (0/1)---- 16.00 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 32.00 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 48.00 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 63.50 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 79.50 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 95.50 KiB/1.08 MiB                    
⠙ Preparing packages... (0/1)---- 111.50 KiB/1.08 MiB                   
⠙ Preparing packages... (0/1)---- 127.50 KiB/1.08 MiB                   
⠙ Preparing packages... (0/1)---- 143.50 KiB/1.08 MiB                   
⠙ Preparing packages... (0/1)---- 159.50 KiB/1.08 MiB                   
⠙ Preparing packages... (0/1)---- 175.50 KiB/1

In [6]:
import litellm
from openai import OpenAI

litellm.completion(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Hello, world!"}],
    client=OpenAI(
        base_url="https://lw-api-management.azure-api.net/lw-ai-api/models",
        api_key="",
        default_query={"api-version": "2024-05-01-preview"},
        default_headers={
            "api-key": "0316bb547e184feab63abe9bbed88214",
        },
    )
)

ModelResponse(id='chatcmpl-COKzoUp4dnytYmFmYvPrjEQttpwyO', created=1759916616, model='gpt-4o-mini-2024-07-18', object='chat.completion', system_fingerprint='fp_efad92c60b', choices=[Choices(finish_reason='stop', index=0, message=Message(content='Hello! How can I assist you today?', role='assistant', tool_calls=None, function_call=None, provider_specific_fields={'refusal': None}, annotations=[]), provider_specific_fields={'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}})], usage=Usage(completion_tokens=10, prompt_tokens=11, total_tokens=21, completion_tokens_details=CompletionTokensDetailsWrapper(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_pr

In [2]:
import os
from langevals import expect
from langevals_langevals.llm_boolean import (
    CustomLLMBooleanEvaluator,
    CustomLLMBooleanSettings,
)
import pandas as pd
import pytest

from agno.agent import Agent
from agno.models.openai.like import OpenAILike

entries = pd.DataFrame(
    {
        "input": [
            "Provide a comprehensive investment analysis for Apple (AAPL).",
            "What is the current stock price of MSFT",
        ],
    }
)

# System instructions for the agent
system_instructions = [
    "Collaborate to provide comprehensive financial and investment insights",
    "Consider both fundamental analysis and market sentiment",
    "Provide actionable investment recommendations with clear rationale",
    "Use tables and charts to display data clearly and professionally",
    "Ensure all claims are supported by data and sources",
    "Present findings in a structured, easy-to-follow format",
    "Only output the final consolidated analysis, not individual agent responses",
    "Dont use emojis",
]


# @pytest.mark.parametrize("entry", entries.itertuples())
# @pytest.mark.flaky(max_runs=3)
@pytest.mark.pass_rate(0.8)
def test_finance_team_agent(input):

    endpoint = "https://lw-api-management.azure-api.net/lw-ai-api/models"
    model_name = "gpt-4o-mini"
    api_version = "2024-05-01-preview"
    subscription_key = "0316bb547e184feab63abe9bbed88214"

    # agent = Agent(
    #     name="Finance Team Test Agent",
    #     model=OpenAILike(
    #         base_url=endpoint,
    #         extra_query={"api-version": api_version},
    #         api_key=subscription_key,
    #         extra_headers={
    #             "api-key": subscription_key,
    #         },
    #         id=model_name,
    #     ),
    #     instructions=system_instructions,
    # )

    # # Run agent
    # response = agent.run(input)
    # output = getattr(response, "content", str(response))
    output = "foo bar"
    print("\n\noutput", output, "\n\n")

    os.environ["X_LITELLM_use_azure_gateway"] = "true"
    os.environ["X_LITELLM_api_base"] = "https://lw-api-management.azure-api.net/lw-ai-api/models"
    os.environ["X_LITELLM_api_version"] = "2024-05-01-preview"
    os.environ["X_LITELLM_azure_api_gateway_header_name"] = "api-key"
    os.environ["X_LITELLM_azure_api_gateway_header_key"] = "0316bb547e184feab63abe9bbed88214"
    os.environ["OPENAI_API_KEY"] = ""

    # Evaluate output
    checker = CustomLLMBooleanEvaluator(
        settings=CustomLLMBooleanSettings(
            prompt="Is the output a comprehensive investment analysis?",
            model="gpt-4o-mini",
        )
    )

    try:
        response = expect(input=input, output=output).to_pass(checker)
        print("Test passed. Response:")
        print(response)
    except AssertionError as e:
        print("Test failed due to AssertionError:")
        print(e)


test_finance_team_agent("Provide a comprehensive investment analysis for Apple (AAPL)")



output foo bar 


Test failed due to AssertionError:
LLM-as-a-Judge Boolean Evaluator to_pass FAILED - The output 'foo bar' does not provide any analysis or relevant information regarding the investment potential, financial performance, risks, or market conditions related to Apple (AAPL). Instead, it appears to be nonsensical and unrelated to the request for a comprehensive investment analysis.
Entry: CustomLLMBooleanEntry(input='Provide a comprehensive investment analysis for Apple (AAPL)', output='foo bar', contexts=None)


In [ ]:
import os
from langevals import expect
from langevals_langevals.llm_boolean import (
    CustomLLMBooleanEvaluator,
    CustomLLMBooleanSettings,
)
import pandas as pd
import pytest

from agno.agent import Agent
from agno.models.openai.like import OpenAILike
from langwatch.evaluations import evaluate
import langwatch

entries = pd.DataFrame(
    {
        "input": [
            "Provide a comprehensive investment analysis for Apple (AAPL).",
            "What is the current stock price of MSFT",
        ],
    }
)

# System instructions for the agent
system_instructions = [
    "Collaborate to provide comprehensive financial and investment insights",
    "Consider both fundamental analysis and market sentiment",
    "Provide actionable investment recommendations with clear rationale",
    "Use tables and charts to display data clearly and professionally",
    "Ensure all claims are supported by data and sources",
    "Present findings in a structured, easy-to-follow format",
    "Only output the final consolidated analysis, not individual agent responses",
    "Dont use emojis",
]


# @pytest.mark.parametrize("entry", entries.itertuples())
# @pytest.mark.flaky(max_runs=3)
@pytest.mark.pass_rate(0.8)
def test_finance_team_agent(input):

    endpoint = "https://lw-api-management.azure-api.net/lw-ai-api/models"
    model_name = "gpt-4o-mini"
    api_version = "2024-05-01-preview"
    subscription_key = "0316bb547e184feab63abe9bbed88214"

    # agent = Agent(
    #     name="Finance Team Test Agent",
    #     model=OpenAILike(
    #         base_url=endpoint,
    #         extra_query={"api-version": api_version},
    #         api_key=subscription_key,
    #         extra_headers={
    #             "api-key": subscription_key,
    #         },
    #         id=model_name,
    #     ),
    #     instructions=system_instructions,
    # )

    # # Run agent
    # response = agent.run(input)
    # output = getattr(response, "content", str(response))
    output = "foo bar"
    print("\n\noutput", output, "\n\n")

    try:
        # response = expect(input=input, output=output).to_pass(checker)
        response = langwatch.evaluations.evaluate(
            "langevals/llm_boolean",
            data={
                "input": input,
                "output": output,
            },
            settings={},
        )
        print("Test passed. Response:")
        print(response)
    except AssertionError as e:
        print("Test failed due to AssertionError:")
        print(e)


test_finance_team_agent("Provide a comprehensive investment analysis for Apple (AAPL)")



output foo bar 


Test passed. Response:
status='error' passed=None score=None details='Evaluator cannot be reached' label=None cost=None error_type='EVALUATOR_ERROR'
